---
title: "Temporal Evolution"
author: "Ali Zaidi"
date: "2025-11-20"
categories: [Data Engineering]
description: "All of that is useful but since we're working with sequence data of a video, we can't leave out changes over time of how these keypoints evolve in the swing"
format:
  html:
    code-fold: true
jupyter: python3
---

1) Velocity of distance
    - how fast two points are moving apart and closer

In [4]:
import numpy as np

def compute_separation_velocity(kp1, kp2, fps=None, time_array=None):
    """
    Computes the rate at which two keypoints are moving apart or closer together.

    Args:
        kp1 (np.ndarray): Shape (N, 2) or (N, 3). Keypoint A coordinates [x, y, (z)].
        kp2 (np.ndarray): Shape (N, 2) or (N, 3). Keypoint B coordinates [x, y, (z)].
        fps (float, optional): Frames per second. Used to scale the velocity to units/second.
        time_array (np.ndarray, optional): Timestamps corresponding to frames.
                                           If provided, overrides fps.

    Returns:
        np.ndarray: Shape (N,). The velocity of distance (separation velocity).
                    Positive = moving apart, Negative = moving closer.
    """
    # 1. Compute Euclidean distance between points at each frame
    # Vectors pointing from kp1 to kp2
    diff_vectors = kp2 - kp1
    
    # Magnitude (L2 norm) of these vectors
    distances = np.linalg.norm(diff_vectors, axis=1)

    # 2. Compute the derivative (gradient) of the distance
    if time_array is not None:
        # If timestamps are provided, use them for non-uniform sampling
        sep_velocity = np.gradient(distances, time_array)
    else:
        # Default to 1.0 if no FPS/time provided (units/frame)
        dt = 1.0 / fps if fps else 1.0
        sep_velocity = np.gradient(distances, dt)

    return sep_velocity

# Example Usage:
# Simulated golf swing  100 frames, 2 keypoints moving apart
N = 100
frames = np.arange(N)
# Keypoint 1 stays at (0,0)
body_kp1 = np.zeros((N, 2))
# Keypoint 2 moves from (1,1) to (10,10)
body_kp2 = np.column_stack((frames * 0.1, frames * 0.1))

velocity = compute_separation_velocity(body_kp1, body_kp2, fps=30)

print(f"Separation Velocity (mean): {np.mean(velocity):.4f} units/sec")


Separation Velocity (mean): 4.2426 units/sec


Key Technical Notes for Golf Analysis <br>
	1.	Noise & Smoothing: Raw pose estimation outputs (like MMPose/YOLO) usually contain high-frequency jitter. Taking the derivative ( np.gradient ) amplifies this noise. It is highly recommended to apply a Savitzky-Golay filter or a Butterworth low-pass filter to the raw keypoints before passing them to this function. <br>
	2.	Vector Projection vs. Derivative: <br>
	-	Derivative Method (Used above): . Simple and robust for general separation analysis. <br>
	-	Projection Method: . This projects the relative velocity vector onto the line connecting the two points. It is mathematically equivalent but requires calculating velocity vectors first, which adds complexity. <br>
	3.	Interpretation: In a golf swing, this metric is useful for “X-Factor” analysis (separation between hips and shoulders) or measuring how quickly the hands move away from the body center during the downswing (radius extension).

2) Velocity of angle
    - Angular speed of a joint -- is the right elbow hitting 90 degrees faster on a 5 swing that a 1 swing?

In [6]:
35*130, 22*2800

(4550, 61600)

3) Acceleration of distance and angle:
    - the second derivative tells you the rate of change of velocity, could be useful at transition points and much more <br>
    --> https://www.perplexity.ai/search/58255efe-03e6-48b0-87c3-0ffdc6c75edf